In [1]:
import sktime
import pandas as pd
import numpy as np

def readcsv(csv_loc:str, name:str):
    csv = pd.read_csv(csv_loc)

    # print(csv.to_string())
    metrics = csv["identifier"].to_list()
    timestamps = csv.columns[1:].to_flat_index()
    timestamps = timestamps.to_numpy().tolist()
    timestamps = pd.Int64Index(timestamps, dtype=np.int64)
    print(timestamps)
    index = pd.MultiIndex.from_product([[name], timestamps], names=['instances','timepoints'])
    print(index)
    vals = csv.drop(labels="identifier",axis=1).to_numpy().transpose()
    s = pd.DataFrame(vals, index=index, columns=metrics)
    return s



simple = readcsv("../Test_datasets/simple1.csv", "simple1")
simple2 = readcsv("../Test_datasets/simple2.csv", "simple1")
from sktime.transformations.panel.rocket import Rocket

rocket = Rocket()
rocket.fit(simple)
rocket.fit(simple2)
simple


C:\Users\nordb\AppData\Local\Temp\ipykernel_8532\2314209265.py:12: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  timestamps = pd.Int64Index(timestamps, dtype=np.int64)
C:\Users\nordb\AppData\Local\Temp\ipykernel_8532\2314209265.py:12: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  timestamps = pd.Int64Index(timestamps, dtype=np.int64)


Int64Index([1, 2, 3, 4, 5, 6, 7, 8], dtype='int64')
MultiIndex([('simple1', 1),
            ('simple1', 2),
            ('simple1', 3),
            ('simple1', 4),
            ('simple1', 5),
            ('simple1', 6),
            ('simple1', 7),
            ('simple1', 8)],
           names=['instances', 'timepoints'])
Int64Index([1, 2, 3, 4, 5, 6, 7, 8], dtype='int64')
MultiIndex([('simple1', 1),
            ('simple1', 2),
            ('simple1', 3),
            ('simple1', 4),
            ('simple1', 5),
            ('simple1', 6),
            ('simple1', 7),
            ('simple1', 8)],
           names=['instances', 'timepoints'])


first  second  third
instances timepoints                      
simple1   1             312       0     45
          2             345       0     87
          3             355       0     23
          4             355       0     67
          5             356       1     12
          6             354       1      3
          7             355       1    654
          8             356       2      4

## Below is my some code I will use for formatting unix timestamps to datetimeIndex, not relevant  ##

In [33]:
#import sktime
import pandas as pd
import numpy as np

def readcsv(csv_loc:str):
    csv = pd.read_csv(csv_loc).transpose()
    metrics = csv.loc["identifier"].to_list()
    timestamps = pd.DatetimeIndex(np.array(csv.index[1:].astype('int').tolist(), dtype='datetime64[s]'))
    print(csv.values[1:].flatten())
    vals = csv.values[1:].flatten()
    index = pd.MultiIndex.from_product([metrics, timestamps], names=['instances','timepoints'])
    s = pd.DataFrame(vals, index=index)
    return s
    # metrics = csv["identifier"].to_list()
    # timestamps = csv.columns[1:].to_flat_index()
    # timestamps = timestamps.to_numpy().tolist()
    # timestamps = pd.Int64Index(timestamps, dtype=np.int64)
    
    # index = pd.MultiIndex.from_product([metrics, timestamps], names=['instances','timepoints'])
    # vals = csv.drop(labels="identifier",axis=1).to_numpy().flatten()
    # s = pd.DataFrame(vals, index=index)
    # return s



simple = readcsv("../DTW/testing.csv")
simple



[0 10 13 0.1 9.0 75.0 0.2 8.0 1.0 0.22 7.0 0.5 0.3 6.0 24.0 0.3 5.0 7.0
 0.3 4.0 5.0 0.4 3.0 23.0 0.5 3.0 56.0]


0
instances timepoints               
increase  2023-01-18 12:32:47     0
          2023-01-18 12:32:52    10
          2023-01-18 12:32:57    13
          2023-01-18 12:33:02   0.1
          2023-01-18 12:33:07   9.0
          2023-01-18 12:33:12  75.0
          2023-01-18 12:33:17   0.2
          2023-01-18 12:33:22   8.0
          2023-01-18 12:33:27   1.0
decrease  2023-01-18 12:32:47  0.22
          2023-01-18 12:32:52   7.0
          2023-01-18 12:32:57   0.5
          2023-01-18 12:33:02   0.3
          2023-01-18 12:33:07   6.0
          2023-01-18 12:33:12  24.0
          2023-01-18 12:33:17   0.3
          2023-01-18 12:33:22   5.0
          2023-01-18 12:33:27   7.0
neither   2023-01-18 12:32:47   0.3
          2023-01-18 12:32:52   4.0
          2023-01-18 12:32:57   5.0
          2023-01-18 12:33:02   0.4
          2023-01-18 12:33:07   3.0
          2023-01-18 12:33:12  23.0
          2023-01-18 12:33:17   0.5
          2023-01-18 12:33:22   3.0
          2023-01-18 12:33:27  56.0

In [23]:
import pandas as pd
import numpy as np

def merge_with_new(existing_vals:pd.DataFrame, new_vals:pd.DataFrame) -> np.ndarray :
    common_columns = existing_vals.columns.intersection(new_vals.columns)
    reduced_existing = existing_vals.reindex(columns=common_columns)
    reduced_new = new_vals.reindex(columns=common_columns)
    timestamps = reduced_existing.index.get_level_values(1)

    row = existing_vals.iloc[(slice(None),0)]
    print(row.index)
    
    newnames = reduced_new.index.get_level_values(0).drop_duplicates()
    oldnames = reduced_existing.index.get_level_values(0).drop_duplicates()
    combined_names = oldnames.union(newnames)

    tuples = [(i,j) for i in combined_names for j in timestamps]
    index = pd.MultiIndex.from_tuples(tuples)

    reduced_combined = pd.concat([reduced_existing,reduced_new],axis=0)
    reduced_combined = reduced_combined.set_index(index)
    #remove any columns that exists in one but not the other
    return reduced_combined

def make_datetime_index(timestamps:list) -> pd.DatetimeIndex:
    beginning = timestamps[0]
    end = timestamps[-1]
    beginning, end = pd.to_datetime((beginning,end), unit="s")
    index = pd.date_range(start=beginning, end=end, periods=len(timestamps))
    return index

def concat_dfs(old:pd.DataFrame, new:pd.DataFrame) -> pd.DataFrame:
    unioned = old.index.union(new.index)
    concated = pd.concat([old.reindex(unioned), new.reindex(unioned)])
    return concated

def readcsv(csv_loc:str, num:int):
    csv = pd.read_csv(csv_loc)
    metrics = csv["identifier"].to_list()
    timestamps = csv.columns[1:].to_flat_index()
    timestamps = timestamps.to_numpy().tolist()
    timestamps = make_datetime_index(timestamps)
    index = pd.MultiIndex.from_product([[num], timestamps], names=['instances','timepoints'])
    vals = csv.drop(labels="identifier",axis=1).to_numpy().transpose()
    s = pd.DataFrame(vals, index=index, columns=metrics)
    return s

def readcsv_modified(csv_loc:str):
    csv = pd.read_csv(csv_loc)
    metrics = csv["identifier"].to_list()
    timestamps = csv.columns[1:].to_flat_index()
    timestamps = timestamps.to_numpy().tolist()
    timestamps = make_datetime_index(timestamps)
   # index = pd.MultiIndex.from_product([[num], timestamps], names=['instances','timepoints'])
    vals = csv.drop(labels="identifier",axis=1).to_numpy().transpose()
    s = pd.DataFrame(vals, index=timestamps, columns=metrics)
    return s

def removeNaNs(df:pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        if df[col].isna().any():
            df = df.drop(col, axis=1)
    return df

def removeUniqueColumns(first:pd.DataFrame, second:pd.DataFrame) -> tuple:
    common_columns = first.columns.intersection(second.columns)
    reduced_old = first.reindex(columns=common_columns)
    reduced_new = second.reindex(columns=common_columns)
    return reduced_old, reduced_new

def removeUniqueColumns2(frames:list) -> list:
    template:pd.DataFrame = frames[0]
    #Round 1: Reduces the first frame to be the smallest size to match with everyone
    for frame in frames[1:]:
        common_columns = template.columns.intersection(frame.columns)
        template = template.reindex(columns=common_columns)
    returnlist = [template]
    #Round 2: Reduces all the other ones to the same minimum
    for frame in frames[1:]:
        common_columns = template.columns.intersection(frame.columns)
        returnlist.append(frame.reindex(columns=common_columns))
    return returnlist

def readcsvs(csv_loc_list:list):
    individual_dataframes = []
    for i in range(len(csv_loc_list)):
        individual_dataframes.append(readcsv_modified(csv_loc_list[i])) #time series

    #Here: Go through the loop once again, start trimming. compare everything to element at 0, trim with it so it stays as the leanest version.

    removed_nans = []
    for frame in individual_dataframes:
        removed_nans.append(removeNaNs(frame))

    # initial_df = removed_nans[0]
    # removed_unique_cols = []
    # for frame in removed_nans[1:]:
    #     reduced_frames = removeUniqueColumns(initial_df, frame)
    #     initial_df = reduced_frames[0]
    #     removed_unique_cols.append(reduced_frames[1])
    
    # removed_unique_cols.append(initial_df)
    removed_unique_cols = removeUniqueColumns2(removed_nans)
    concated = pd.concat(removed_unique_cols, keys=[f'csv {i}' for i in range(1, len(removed_unique_cols)+1)])

    #concated.index = pd.MultiIndex.from_tuples([(idx, date) for idx, date in zip(concated.index.get_level_values(0), concated.index.get_level_values(1))], names=['files','times'])
    concated2 = concated.reindex(index=pd.MultiIndex.from_tuples([(idx, date) for idx, date in zip(concated.index.get_level_values(0), concated.index.get_level_values(1))], names=['files','times']))
    #print(concated2)
    #return removed_unique_cols
    return concated2

def add_to_frame(csv_loc:str, frame:pd.DataFrame, name:str):
    #call readcsv to get a finished dataframe with the things in it. 
    #get an index of potentially different length that has to be cleaned.
    current_highest_num = frame.index.get_level_values('instances').max() 
    newframe = readcsv(csv_loc, name, current_highest_num)

    return concat_dfs(frame, newframe)
    csv = pd.read_csv(csv_loc)
    new_vals = csv.drop(labels="identifier", axis=1).to_numpy().transpose()
    #vals = np.append(frame.values, new_vals)
    vals = np.array(frame.values)
    merged = merge_with_new(vals, new_vals)
    added = np.concatenate([vals, new_vals],axis=0)
    print(added, added.shape)
    #Get a copy of the date index, expand it with another name
    index = frame.index
    name_index = index.levels[0].to_list()
    name_index.append(name)
    timepoints = index.get_level_values('timepoints')
    new_index = pd.MultiIndex.from_product([name_index, timepoints], names=['instances','timepoints'])
    

    df = pd.DataFrame(vals,  index=new_index, columns=frame.columns)
    return df

# catalog = readcsv("../Test_datasets/catalog.csv", "catalog")
# updated = add_to_frame("../Test_datasets/even_load.csv", catalog, "even_load")
# updated = add_to_frame("../Test_datasets/catalog2.csv",updated, "catalog")
# updated
#simple = readcsv("../Test_datasets/simple1.csv", "simple1")
#simple2 = readcsv("../Test_datasets/simple2.csv", "simple1")
# simple
#Do I really have to synchronize the dates? What happens if I just let them be as they are. Does the algorithm care? It would be relative and not absolute anyway. Fuck all this actually. Just concat that shit?
#266 * 601
#Could not broadcast from 133,601.
file_list = ["F:/Master/Kubernetes/sockshop/microservices-demo/analysis/Test_datasets/catalog.csv",
             "F:/Master/Kubernetes/sockshop/microservices-demo/analysis/Test_datasets/catalog2.csv",
             "F:/Master/Kubernetes/sockshop/microservices-demo/analysis/Test_datasets/even_load.csv"]
complete_df = readcsvs(file_list)

from get_all_metrics_with_tags import get_all_metrics_with_tags
better_file_list = get_all_metrics_with_tags(r"F:\Master\Kubernetes\sockshop\microservices-demo\query\automated\generated_csvs")
y = ["catalog",
     "catalog",
     "even_load"]
#print(complete_df)
from sklearn.preprocessing import StandardScaler
from sktime.transformations.panel.rocket import Rocket
scaler = StandardScaler()
fitted = scaler.fit_transform(complete_df)
rocket = Rocket()
rocket.fit(fitted,y)
new_df = pd.DataFrame(data=fitted, index = complete_df.index, columns=complete_df.columns)


195510 195510
